In [1]:
import torch.nn as nn

C:\Users\USER\anaconda3\envs\env\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\USER\anaconda3\envs\env\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\USER\anaconda3\envs\env\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
C:\Users\USER\anaconda3\envs\env\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
class CharCNN(nn.Module):
    def __init__(self,                          # CharCNN 클래스의 인스턴스를 생성했을 때 갖게 되는 성질을 정의하는 메서드
                max_word_len = 10,
                kernel_lst = "2,3,4",
                num_filters=32,
                char_vocab_size = 1000,
                char_emb_dim = 30,
                final_char_dim = 50
                ):
        super(CharCNN, self).__init__()         #nn.Module 내에 있는 메서드를 상속받아 이용한다.

        # Initialize character embedding
        


SyntaxError: unexpected EOF while parsing (<ipython-input-2-2e71fa027d43>, line 1)